In [1]:
path = "pretrained_models/Wan2.1-T2V-1.3B/"

In [2]:
import torch
from diffsynth import ModelManager, WanVideoPipeline

# Load models
model_manager = ModelManager(device="cpu")
model_manager.load_models(
    [
        f"{path}/diffusion_pytorch_model.safetensors",
        f"{path}/models_t5_umt5-xxl-enc-bf16.pth",
        f"{path}/Wan2.1_VAE.pth",
    ],
    torch_dtype=torch.float8_e4m3fn,  # You can set `torch_dtype=torch.float8_e4m3fn` to enable FP8 quantization.
)
pipe = WanVideoPipeline.from_model_manager(model_manager, torch_dtype=torch.bfloat16, device="cuda")
pipe.enable_vram_management(num_persistent_param_in_dit=None)

/home/ubuntu/dev/wan-api/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading models from: pretrained_models/Wan2.1-T2V-1.3B//diffusion_pytorch_model.safetensors
    model_name: wan_video_dit model_class: WanModel
        This model is initialized with extra kwargs: {'has_image_input': False, 'patch_size': [1, 2, 2], 'in_dim': 16, 'dim': 1536, 'ffn_dim': 8960, 'freq_dim': 256, 'text_dim': 4096, 'out_dim': 16, 'num_heads': 12, 'num_layers': 30, 'eps': 1e-06}
    The following models are loaded: ['wan_video_dit'].
Loading models from: pretrained_models/Wan2.1-T2V-1.3B//models_t5_umt5-xxl-enc-bf16.pth
    model_name: wan_video_text_encoder model_class: WanTextEncoder
    The following models are loaded: ['wan_video_text_encoder'].
Loading models from: pretrained_models/Wan2.1-T2V-1.3B//Wan2.1_VAE.pth
    model_name: wan_video_vae model_class: WanVideoVAE
    The following models are loaded: ['wan_video_vae'].
Using wan_video_text_encoder from pretrained_models/Wan2.1-T2V-1.3B//models_t5_umt5-xxl-enc-bf16.pth.
Using wan_video_dit from pretrained_models/Wan2.

In [3]:
from diffsynth import save_video

# Text-to-video
video = pipe(
    prompt="纪实摄影风格画面，一只活泼的小狗在绿茵茵的草地上迅速奔跑。小狗毛色棕黄，两只耳朵立起，神情专注而欢快。阳光洒在它身上，使得毛发看上去格外柔软而闪亮。背景是一片开阔的草地，偶尔点缀着几朵野花，远处隐约可见蓝天和几片白云。透视感鲜明，捕捉小狗奔跑时的动感和四周草地的生机。中景侧面移动视角。",
    negative_prompt="色调艳丽，过曝，静态，细节模糊不清，字幕，风格，作品，画作，画面，静止，整体发灰，最差质量，低质量，JPEG压缩残留，丑陋的，残缺的，多余的手指，画得不好的手部，画得不好的脸部，畸形的，毁容的，形态畸形的肢体，手指融合，静止不动的画面，杂乱的背景，三条腿，背景人很多，倒着走",
    num_inference_steps=50,
    seed=0,
    tiled=True,
    # TeaCache parameters
    tea_cache_l1_thresh=0.05,  # The larger this value is, the faster the speed, but the worse the visual quality.
    tea_cache_model_id="Wan2.1-T2V-1.3B",  # Choose one in (Wan2.1-T2V-1.3B, Wan2.1-T2V-14B, Wan2.1-I2V-14B-480P, Wan2.1-I2V-14B-720P).
)
save_video(video, "video-1.mp4", fps=15, quality=5)

Saving video:   0%|          | 0/81 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Saving video: 100%|██████████| 81/81 [00:00<00:00, 127.51it/s]


In [4]:
from IPython.display import Video

Video("video-1.mp4")